# Setup API keys

- OpenAI: https://openai.com/api/
- Anthropic: https://console.anthropic.com/
- Google: https://ai.google.dev/gemini-api/

In [ ]:
import os, sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display

In [ ]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    print("OpenAI API key not loaded.")
    sys.exit(1)
else:
    print("OpenAI API key loaded.")

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
if not GEMINI_API_KEY:
    print("Gemini API key not loaded.")
    sys.exit(1)
else:
    print("Gemini API key loaded.")

ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
if not ANTHROPIC_API_KEY:
    print("Anthropic API key not loaded.")
    sys.exit(1)
else:
    print("Anthropic API key loaded.")

In [ ]:
openai = OpenAI()
gemini = google.generativeai.configure()
claude = anthropic.Anthropic()

In [ ]:
system_prompt = "You are an assistant that is great at telling dank and super dark jokes"
user_prompt = "Tell me a joke for pianists"

prompts = [
    { "role": "system", "content": system_prompt },
    { "role": "user", "content": user_prompt }
]

In [ ]:
for m in ["gpt-3.5-turbo", "gpt-4o-mini", "gpt-4o"]:
    completion = openai.chat.completions.create(
        model = m,
        messages = prompts,
        temperature = 0.7
    )
    print(f'===== {m} =====\n')
    print(completion.choices[0].message.content)
    print("================\n")

In [ ]:
message = claude.messages.create(
    model = "claude-3-7-sonnet-20250219",
    max_tokens = 200,
    temperature = 0.7,
    system = system_prompt,
    messages = [
        { "role": "user", "content": user_prompt }
    ]
)
print(message.content[0].text)

In [ ]:
# Stream

response = claude.messages.stream(
    model = "claude-3-7-sonnet-20250219",
    max_tokens = 200,
    temperature = 0.7,
    system = system_prompt,
    messages = [
        { "role": "user", "content": user_prompt }
    ]
)
with response as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)

In [ ]:
gemini = google.generativeai.GenerativeModel(
    model_name = "gemini-2.5-flash",
    system_instruction = system_prompt
)
response = gemini.generate_content(user_prompt)
print(response.text)

# Adversarial Conversation Between gpt-4o-mini and claude-3-haiku-20240307

Let the bots discuss

NOTE: This is a bit different implementation from the course.

In [ ]:
def gpt_convo(gpt_messages = ["Hi"], claude_messages = ["Hi"]):
    messages = [
        {"role": "system", "content": "You are very argumentative. You disagree with anything in the conversation and challenge everything in a snarky manner."}
    ]
    # Merge the messages
    for gpt_msg, claude_msg in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt_msg})
        messages.append({"role": "user", "content": claude_msg})
    # Request to GPT
    completion = openai.chat.completions.create(
            model = "gpt-4o-mini",
            messages = messages
    )
    return completion.choices[0].message.content

def claude_convo(claude_messages = ["Hi"], gpt_messages = ["Hi"]):
    claude_system_prompt = "You are very polite and courteous. You try to agree with what the other person says or find common ground. If they are argumentative you try to calm them down and chat with them."
    messages = []
    # Merge the messages
    for gpt_msg, claude_msg in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt_msg})
        messages.append({"role": "assistant", "content": claude_msg})
    # Start with claude
    messages.append({"role": "user", "content": gpt_messages[-1]})
    # Request to Claude
    message = claude.messages.create(
        model = "claude-3-haiku-20240307",
        system = claude_system_prompt,
        messages = messages,
        max_tokens = 500
    )
    return message.content[0].text

In [ ]:
gpt_convo()

In [ ]:
claude_convo()

In [ ]:
gpt_messages = ["Hi"]
claude_messages = ["Hi"]
for i in range(5):
    gpt_next = gpt_convo(gpt_messages = gpt_messages, claude_messages = claude_messages)
    print(f"GPT > {gpt_messages[-1]}\n", flush=True)
    gpt_messages.append(gpt_next)

    claude_next = claude_convo(claude_messages = claude_messages, gpt_messages = gpt_messages)
    print(f"Claude > {claude_messages[-1]}\n", flush=True)
    claude_messages.append(claude_next)